In [1]:
import sqlite3 
import pandas as pd

In [2]:
df_globalstore = pd.read_excel('Global Superstore.xls') 
df_globalstore.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,...,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority
0,32298,CA-2012-124891,2012-07-31,2012-07-31,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,...,TEC-AC-10003033,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wir...,2309.650,7,0.0,762.1845,933.57,Critical
1,26341,IN-2013-77878,2013-02-05,2013-02-07,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,...,FUR-CH-10003950,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.7650,923.63,Critical
2,25330,IN-2013-71249,2013-10-17,2013-10-18,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,...,TEC-PH-10004664,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.9710,915.49,Medium
3,13524,ES-2013-1579342,2013-01-28,2013-01-30,First Class,KM-16375,Katherine Murray,Home Office,Berlin,Berlin,...,TEC-PH-10004583,Technology,Phones,"Motorola Smart Phone, Cordless",2892.510,5,0.1,-96.5400,910.16,Medium
4,47221,SG-2013-4320,2013-11-05,2013-11-06,Same Day,RH-9495,Rick Hansen,Consumer,Dakar,Dakar,...,TEC-SHA-10000501,Technology,Copiers,"Sharp Wireless Fax, High-Speed",2832.960,8,0.0,311.5200,903.04,Critical


In [3]:
#renaming columns to lower case and removing spaces inbetween
df_globalstore.columns = df_globalstore.columns.str.replace(' ', '_').str.lower()
df_globalstore.columns = df_globalstore.columns.str.replace('-', '_').str.lower()

In [4]:
#creating a price column for this exercise purpose

df_globalstore['price'] = df_globalstore['sales']/df_globalstore['quantity']
df_globalstore.head()

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,city,state,...,category,sub_category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority,price
0,32298,CA-2012-124891,2012-07-31,2012-07-31,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,...,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wir...,2309.650,7,0.0,762.1845,933.57,Critical,329.950
1,26341,IN-2013-77878,2013-02-05,2013-02-07,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,...,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.7650,923.63,Critical,412.155
2,25330,IN-2013-71249,2013-10-17,2013-10-18,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,...,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.9710,915.49,Medium,575.019
3,13524,ES-2013-1579342,2013-01-28,2013-01-30,First Class,KM-16375,Katherine Murray,Home Office,Berlin,Berlin,...,Technology,Phones,"Motorola Smart Phone, Cordless",2892.510,5,0.1,-96.5400,910.16,Medium,578.502
4,47221,SG-2013-4320,2013-11-05,2013-11-06,Same Day,RH-9495,Rick Hansen,Consumer,Dakar,Dakar,...,Technology,Copiers,"Sharp Wireless Fax, High-Speed",2832.960,8,0.0,311.5200,903.04,Critical,354.120


Examples of possible tables:

orders
sale related columns
table id columns

category
id_category
name

subcategory
id_subcategory
id_category
name

product
id_product
id_subcategory
name_product

cities
id_city
name_city
state
country
postalcode
region
market



1) - Utilizando Funções e Pipelines para preparação dos dados

In [5]:
#transforming columns in categorical and adding related id column to dataset
def retornaCatCodes(df, coluna):
  print(coluna)
  df = df.astype({coluna:'category'})
  newcol = coluna+'_id'
  df[newcol] = df[coluna].cat.codes.apply(lambda x: x + 1)
  return df ,newcol

#creating a new dataframe

def criarDf(df_name, colunas):
    df1 = df_name[colunas]
    return df1.drop_duplicates(subset = [colunas[0]])

#simple to_sql function

def toSql(df,name):
    df.to_sql(name, bd, index=False)
    return df

In [6]:
#pipeline to execute the first two functions

def pipeline(df, colunas):
    df_original, newcol = retornaCatCodes(df,colunas[0])
    colunas.insert(0,newcol)
    df = criarDf(df_original, colunas)
    return df,df_original

In [7]:
df_city, df_globalstore = pipeline(df_globalstore,['city','state','country','region','postal_code'])
df_category,df_globalstore = pipeline(df_globalstore,['category'])
df_sub_category,df_globalstore = pipeline(df_globalstore,['sub_category','category_id'])
df_product,df_globalstore = pipeline(df_globalstore,['product_name','sub_category_id','price'])
df_customer,df_globalstore = pipeline(df_globalstore,['customer_name','segment'])
df_ship_mode,df_globalstore = pipeline(df_globalstore,['ship_mode'])
df_order_priority,df_globalstore = pipeline(df_globalstore,['order_priority'])

city
category
sub_category
product_name
customer_name
ship_mode
order_priority


In [8]:
df_globalstore

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,city,state,...,shipping_cost,order_priority,price,city_id,category_id,sub_category_id,product_name_id,customer_name_id,ship_mode_id,order_priority_id
0,32298,CA-2012-124891,2012-07-31,2012-07-31,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,...,933.570,Critical,329.950,2291,3,1,2751,633,2,1
1,26341,IN-2013-77878,2013-02-05,2013-02-07,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,...,923.630,Critical,412.155,3519,1,6,2526,414,3,1
2,25330,IN-2013-71249,2013-10-17,2013-10-18,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,...,915.490,Medium,575.019,498,3,14,2503,182,1,4
3,13524,ES-2013-1579342,2013-01-28,2013-01-30,First Class,KM-16375,Katherine Murray,Home Office,Berlin,Berlin,...,910.160,Medium,578.502,376,3,14,2415,425,1,4
4,47221,SG-2013-4320,2013-11-05,2013-11-06,Same Day,RH-9495,Rick Hansen,Consumer,Dakar,Dakar,...,903.040,Critical,354.120,858,3,7,3159,633,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51285,29002,IN-2014-62366,2014-06-19,2014-06-19,Same Day,KE-16420,Katrina Edelman,Corporate,Kure,Hiroshima,...,0.010,Medium,13.020,1727,2,9,240,428,2,4
51286,35398,US-2014-102288,2014-06-20,2014-06-24,Standard Class,ZC-21910,Zuschuss Carroll,Consumer,Houston,Texas,...,0.010,Medium,0.444,1401,2,2,1923,794,4,4
51287,40470,US-2013-155768,2013-12-02,2013-12-02,Same Day,LB-16795,Laurel Beltran,Home Office,Oxnard,California,...,0.010,High,7.640,2409,2,8,6,453,2,2
51288,9596,MX-2012-140767,2012-02-18,2012-02-22,Standard Class,RB-19795,Ross Baird,Home Office,Valinhos,São Paulo,...,0.003,Medium,6.720,3347,2,4,88,653,4,4


In [9]:
#reordering df_globalstore columns
df_globalstore = df_globalstore[['row_id', 'order_id', 'customer_id', 'product_name_id','ship_mode_id', 'order_priority_id', 'city_id', 'order_date', 'ship_date', 'sales', 'quantity', 'shipping_cost', 'discount', 'profit']]

In [10]:
bd = sqlite3.connect('superstore2.db')

In [11]:
toSql(df_globalstore,'orders')
toSql(df_city,'city')
toSql(df_category,'category')
toSql(df_sub_category,'sub_category')
toSql(df_product,'product')
toSql(df_customer,'customer')
toSql(df_ship_mode,'ship_mode')
toSql(df_order_priority,'order_priority')

,order_priority_id,order_priority
0,1,Critical
2,4,Medium
7,2,High
8,3,Low


In [12]:
def executaQuery(query):
  return pd.read_sql(query,bd)

In [13]:
executaQuery('SELECT * FROM orders')

,row_id,order_id,customer_id,product_name_id,ship_mode_id,order_priority_id,city_id,order_date,ship_date,sales,quantity,shipping_cost,discount,profit
0,32298,CA-2012-124891,RH-19495,2751,2,1,2291,2012-07-31 00:00:00,2012-07-31 00:00:00,2309.650,7,933.570,0.0,762.1845
1,26341,IN-2013-77878,JR-16210,2526,3,1,3519,2013-02-05 00:00:00,2013-02-07 00:00:00,3709.395,9,923.630,0.1,-288.7650
2,25330,IN-2013-71249,CR-12730,2503,1,4,498,2013-10-17 00:00:00,2013-10-18 00:00:00,5175.171,9,915.490,0.1,919.9710
3,13524,ES-2013-1579342,KM-16375,2415,1,4,376,2013-01-28 00:00:00,2013-01-30 00:00:00,2892.510,5,910.160,0.1,-96.5400
4,47221,SG-2013-4320,RH-9495,3159,2,1,858,2013-11-05 00:00:00,2013-11-06 00:00:00,2832.960,8,903.040,0.0,311.5200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51285,29002,IN-2014-62366,KE-16420,240,2,4,1727,2014-06-19 00:00:00,2014-06-19 00:00:00,65.100,5,0.010,0.0,4.5000
51286,35398,US-2014-102288,ZC-21910,1923,4,4,1401,2014-06-20 00:00:00,2014-06-24 00:00:00,0.444,1,0.010,0.8,-1.1100
51287,40470,US-2013-155768,LB-16795,6,2,2,2409,2013-12-02 00:00:00,2013-12-02 00:00:00,22.920,3,0.010,0.0,11.2308
51288,9596,MX-2012-140767,RB-19795,88,4,4,3347,2012-02-18 00:00:00,2012-02-22 00:00:00,13.440,2,0.003,0.0,2.4000


1) Listing all sales with products priced above 500:

In [14]:
executaQuery('SELECT * FROM orders JOIN product ON orders.product_name_id = product.product_name_id WHERE product.price > 500')

,row_id,order_id,customer_id,product_name_id,ship_mode_id,order_priority_id,city_id,order_date,ship_date,sales,quantity,shipping_cost,discount,profit,product_name_id,product_name,sub_category_id,price
0,1947,MX-2014-147480,EB-13750,2503,2,2,2907,2014-10-02 00:00:00,2014-10-02 00:00:00,1703.7600,4,131.0400,0.00,681.4400,2503,"Nokia Smart Phone, with Caller ID",14,575.019
1,1961,MX-2013-153479,SC-20230,2503,4,4,2637,2013-03-20 00:00:00,2013-03-24 00:00:00,1277.8200,3,81.6650,0.00,511.0800,2503,"Nokia Smart Phone, with Caller ID",14,575.019
2,1990,US-2013-164119,TC-20980,2503,4,4,3179,2013-11-15 00:00:00,2013-11-21 00:00:00,1022.2560,4,110.3650,0.40,-0.0640,2503,"Nokia Smart Phone, with Caller ID",14,575.019
3,2383,MX-2014-117352,AH-10210,2503,3,4,2139,2014-02-04 00:00:00,2014-02-08 00:00:00,2981.5800,7,180.5310,0.00,1192.5200,2503,"Nokia Smart Phone, with Caller ID",14,575.019
4,6915,US-2014-159884,BT-11530,2503,4,4,518,2014-11-28 00:00:00,2014-12-03 00:00:00,1277.8200,5,129.1530,0.40,-0.0800,2503,"Nokia Smart Phone, with Caller ID",14,575.019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749,1463,US-2014-154991,NC-18535,626,3,2,595,2014-10-18 00:00:00,2014-10-23 00:00:00,107.0160,1,2.5500,0.70,-196.2040,626,"Bevis Wood Table, Fully Assembled",17,535.080
750,10259,US-2013-128643,PO-18850,626,4,4,2556,2013-08-12 00:00:00,2013-08-16 00:00:00,142.6880,1,6.3700,0.60,-160.5320,626,"Bevis Wood Table, Fully Assembled",17,535.080
751,26583,ID-2013-30544,CL-12700,626,1,2,1385,2013-06-13 00:00:00,2013-06-15 00:00:00,460.1688,2,80.9800,0.57,-117.7512,626,"Bevis Wood Table, Fully Assembled",17,535.080
752,46760,UP-2014-830,NG-8430,626,3,2,1941,2014-12-13 00:00:00,2014-12-17 00:00:00,1070.1600,2,116.6900,0.00,321.0000,626,"Bevis Wood Table, Fully Assembled",17,535.080


2) Listing producs from the category Technology with price above 500:

In [16]:
executaQuery("SELECT * FROM orders JOIN product ON orders.product_name_id = product.product_name_id \
             JOIN sub_category ON product.sub_category_id = sub_category.sub_category_id  \
             JOIN category ON sub_category.category_id = category.category_id \
             WHERE product.price > 500 AND category.category = 'Technology'")

,row_id,order_id,customer_id,product_name_id,ship_mode_id,order_priority_id,city_id,order_date,ship_date,sales,...,profit,product_name_id,product_name,sub_category_id,price,sub_category_id,sub_category,category_id,category_id,category
0,31812,CA-2014-127432,AD-10180,848,4,4,1231,2014-01-23 00:00:00,2014-01-28 00:00:00,2999.950,...,1379.9770,848,Canon Image Class D660 Copier,7,599.990,7,Copiers,3,3,Technology
1,40117,CA-2014-100622,DK-13090,848,4,4,2383,2014-11-04 00:00:00,2014-11-08 00:00:00,959.984,...,311.9948,848,Canon Image Class D660 Copier,7,599.990,7,Copiers,3,3,Technology
2,32530,CA-2013-153682,BG-11695,849,1,1,732,2013-05-31 00:00:00,2013-06-02 00:00:00,839.988,...,69.9990,849,Canon Imageclass D680 Copier / Fax,7,559.992,7,Copiers,3,3,Technology
3,32846,CA-2011-134278,EP-13915,849,1,4,2291,2011-07-06 00:00:00,2011-07-08 00:00:00,559.992,...,174.9975,849,Canon Imageclass D680 Copier / Fax,7,559.992,7,Copiers,3,3,Technology
4,34352,US-2012-100377,TS-21370,849,4,4,695,2012-08-28 00:00:00,2012-09-01 00:00:00,2799.960,...,874.9875,849,Canon Imageclass D680 Copier / Fax,7,559.992,7,Copiers,3,3,Technology
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,43185,AO-2012-5450,LH-6900,2992,2,2,1926,2012-11-12 00:00:00,2012-11-12 00:00:00,636.150,...,311.7000,2992,"Samsung Smart Phone, with Caller ID",14,572.535,14,Phones,3,3,Technology
396,46694,IR-2011-3170,FH-4365,2992,4,3,3180,2011-07-25 00:00:00,2011-07-31 00:00:00,636.150,...,311.7000,2992,"Samsung Smart Phone, with Caller ID",14,572.535,14,Phones,3,3,Technology
397,47985,MO-2012-100,AJ-960,2992,4,2,614,2012-08-27 00:00:00,2012-08-31 00:00:00,2544.600,...,1246.8000,2992,"Samsung Smart Phone, with Caller ID",14,572.535,14,Phones,3,3,Technology
398,50436,IZ-2014-6400,Dl-3600,2992,1,1,249,2014-09-11 00:00:00,2014-09-13 00:00:00,636.150,...,311.7000,2992,"Samsung Smart Phone, with Caller ID",14,572.535,14,Phones,3,3,Technology


Checking how much each category sold in products with price above 500:


In [17]:
executaQuery(" SELECT category, COUNT(category) as qtd \
             FROM orders JOIN product ON orders.product_name_id = product.product_name_id \
             JOIN sub_category ON product.sub_category_id = sub_category.sub_category_id \
             JOIN category ON sub_category.category_id = category.category_id \
             WHERE product.price > 500\
             GROUP BY category\
            ")

,category,qtd
0,Furniture,126
1,Office Supplies,228
2,Technology,400
